In [16]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # BE QUIET!!!! (info and warnings are not printed)
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

os.environ["CUDA_VISIBLE_DEVICES"] = "1" for 2080 Ti
 

In [17]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() #compatibility
np.random.seed(1)
tf.set_random_seed(1)
# from tools import problems,networks,train
import math
import sys
import numpy.linalg as la

# from sparc_sim import sparc_sim
# from sparc_se import sparc_se
import matplotlib.pyplot as plt
import time

Instructions for updating:
non-resource variables are not supported in the long term


In [18]:
"""
M = Number of columns in a section 
L = Number of section
R = Rate
pnz = sparsity
awgn_var = AWGN channel variance
"""

P, M, L, R, pnz, awgn_var = 15, 32, 50, 0.5, 0.1, 0.01       #try with L=1000 sections later, changed R from 1.3 to 1.2
K = 1    # for unmodulated case

'''
code_params   = {'P': 15.0,    # Average codeword symbol power constraint
                 'R': 1.3,     # Rate
                 'L': 100,    # Number of sections
                 'M': 32}      # Columns per section
'''                 

"\ncode_params   = {'P': 15.0,    # Average codeword symbol power constraint\n                 'R': 1.3,     # Rate\n                 'L': 100,    # Number of sections\n                 'M': 32}      # Columns per section\n"

## Generate Message vector from bit stream 

In [19]:
# minimum value for no_cols is 2 and should be even
rng = np.random.RandomState(seed=None) 

def gen_msg(no_cols,M, L, K, R, rng):
    #no_cols = 2000
    colsby2 = int(np.divide(no_cols,2))

    beta1 = np.zeros((L*M,colsby2))
    beta_val = np.zeros((L*M, colsby2) )

    bit_len = int(round(L*np.log2(K*M)))
    logM = int(round(np.log2(M)))
    sec_size = logM
    L = bit_len // sec_size

    W = np.array(P)

    for i in range(1,no_cols):
        #bit_len = int(round(L*np.log2(K*M)))   
        bits_in = rng.randint(2, size=bit_len)  #generates a boolean vector of length 5000  (removed dtype=' bool')

        beta0 = np.zeros(L*M)
        for l in range(L):
            bits_sec = bits_in[l*sec_size : l*sec_size + logM]
            assert 0 < logM < 64
            idx = bits_sec.dot(1 << np.arange(logM)[::-1])  # idx = decimal equivalent of the 5 bits (eg: 10100 will give 20)
            beta0[l*M + idx] = 1      # will make a 1 at the decimal equivalent in the l-th section

    # beta0 is a M*L length message vector that has 1 at decimal values of (5 bit stream) 32 length section.   

        n = int(round(bit_len/R))
        R_actual = bit_len / n      # Actual rate

        #beta0 = beta0.reshape((L*M),1)
        if i <= colsby2:
            beta1[:,i-1] = beta0
        else:
            beta_val[:,i - no_cols] = beta0    

    return beta1, beta_val, n, colsby2       

In [20]:
beta1, beta_val, n, colsby2 = gen_msg(2000,M, L, K, R, rng)

In [21]:
print(beta1[:,1].sum())

50.0


## Generate Measurement Matrix
Right now, we are working with Random i.i.d gaussian matrices with N(0,1/n). Later we may use FFT/DCT matrices for better computation. 

In [22]:
A = np.random.normal(size=(n, L*M), scale=1.0 / math.sqrt(n)).astype(np.float32)
A_ = tf.constant(A,name='A',dtype = tf.float32 )  #Made a tf constant since it need not be trained

beta1 is for training, beta_val is for validation. 

In [23]:
xgen_ = tf.constant(beta1, dtype= tf.float32)  #changed to tf.constant
#xgen_ = tf.cast(beta1, dtype= tf.float32)

ygen_ = tf.matmul(A_,xgen_) + tf.random.normal( (n,colsby2),stddev=math.sqrt( awgn_var ) )
y_val = np.matmul(A, beta_val) + np.random.normal( size = (n,colsby2),scale= math.sqrt(awgn_var)) #tf.sqrt(awgn_var)*rng.randn(n)

xgen_, ygen_ is given for training
beta_val, y_val is for validation

In [24]:
print(xgen_)
print(ygen_)
print(A_)

Tensor("Const:0", shape=(1600, 1000), dtype=float32)
Tensor("add:0", shape=(500, 1000), dtype=float32)
Tensor("A:0", shape=(500, 1600), dtype=float32)


## BUILD LISTA

In [25]:
T, initial_lambda = 6, 0.1

In [26]:
def simple_soft_threshold(r_, lam_):
    "implement a soft threshold function y=sign(r)*max(0,abs(r)-lam)"
    lam_ = tf.maximum(lam_, 0)
    return tf.sign(r_) * tf.maximum(tf.abs(r_) - lam_, 0)

In [27]:
# creating placeholders, which will hold dataset later.
x_ = tf.compat.v1.placeholder( tf.float32,(L*M,None),name='x' )
y_ = tf.compat.v1.placeholder( tf.float32,(n,None),name='y' )

In [28]:
eta = simple_soft_threshold #creating an object
B = A.T / (1.01 * la.norm(A,2)**2) #(This is according to original code, not sure why)
#B = A.T #numpy array

# All of these below are tf, which means they will be stored as graphs and will run later. 
# B and S are the trainable parameters
B_ = tf.Variable(B,dtype=tf.float32,name='B_0') #creating tf.variable to make it a trainable parameter. 
S_ = tf.Variable(np.identity(L*M) - np.matmul(B, A),dtype=tf.float32,name='S_0' )

By_ = tf.matmul(B_,y_)  # This will be the input for the shrinkage function in the first iteration

In [29]:
print(B_)
print(By_)

<tf.Variable 'B_0:0' shape=(1600, 500) dtype=float32_ref>
Tensor("MatMul_1:0", shape=(1600, ?), dtype=float32)


In [30]:
# not sure why it is written as a list. Might get clear later.
layers = []
layers.append( ('Linear',By_,None) ) 

In [31]:
initial_lambda = np.array(initial_lambda).astype(np.float32)
lam0_ = tf.Variable(initial_lambda,name='lam_0')  #to make it learnable

xhat_ = eta( By_, lam0_) #first itertion and xhat_0 is all 0s
layers.append( ('LISTA T=1',xhat_, (lam0_,) ) )

for t in range(1,T):
    lam_ = tf.Variable( initial_lambda,name='lam_{0}'.format(t) ) # using the same lambda
    xhat_ = eta( tf.matmul(S_,xhat_) + By_, lam_ )  # will be stored as graphs which will run during a session later.   
    layers.append( ('LISTA T='+str(t+1),xhat_,(lam_,)) ) # creating layers (xhat_ is an operation)         

In [32]:
print(*layers , sep = "\n")
print(lam0_)

('Linear', <tf.Tensor 'MatMul_1:0' shape=(1600, ?) dtype=float32>, None)
('LISTA T=1', <tf.Tensor 'mul:0' shape=(1600, ?) dtype=float32>, (<tf.Variable 'lam_0:0' shape=() dtype=float32_ref>,))
('LISTA T=2', <tf.Tensor 'mul_1:0' shape=(1600, ?) dtype=float32>, (<tf.Variable 'lam_1:0' shape=() dtype=float32_ref>,))
('LISTA T=3', <tf.Tensor 'mul_2:0' shape=(1600, ?) dtype=float32>, (<tf.Variable 'lam_2:0' shape=() dtype=float32_ref>,))
('LISTA T=4', <tf.Tensor 'mul_3:0' shape=(1600, ?) dtype=float32>, (<tf.Variable 'lam_3:0' shape=() dtype=float32_ref>,))
('LISTA T=5', <tf.Tensor 'mul_4:0' shape=(1600, ?) dtype=float32>, (<tf.Variable 'lam_4:0' shape=() dtype=float32_ref>,))
('LISTA T=6', <tf.Tensor 'mul_5:0' shape=(1600, ?) dtype=float32>, (<tf.Variable 'lam_5:0' shape=() dtype=float32_ref>,))
<tf.Variable 'lam_0:0' shape=() dtype=float32_ref>


In [33]:
for name,xhat_,var_list in layers:
    print(xhat_)

Tensor("MatMul_1:0", shape=(1600, ?), dtype=float32)
Tensor("mul:0", shape=(1600, ?), dtype=float32)
Tensor("mul_1:0", shape=(1600, ?), dtype=float32)
Tensor("mul_2:0", shape=(1600, ?), dtype=float32)
Tensor("mul_3:0", shape=(1600, ?), dtype=float32)
Tensor("mul_4:0", shape=(1600, ?), dtype=float32)
Tensor("mul_5:0", shape=(1600, ?), dtype=float32)


## Setup Training LISTA

In [34]:
trinit,refinements,final_refine = 1e-3, (.5,.1,.01), None
losses_=[]
nmse_=[]
trainers_=[]

"""assert is used when, If the statement is true then it does nothing and continues 
the execution, but if the statement is False then it stops the execution of the program 
and throws an error.
"""
assert np.array(refinements).min()>0,'all refinements must be in (0,1]'
assert np.array(refinements).max()<=1,'all refinements must be in (0,1]'

tr_ = tf.Variable(trinit,name='tr',trainable=False) #Learning rate
training_stages=[]

In [35]:
tf.disable_v2_behavior() #compatibility

In [36]:
nmse_denom_ = tf.nn.l2_loss(x_)  #computes half of l2 loss without the sqrt

In [37]:
for name,xhat_,var_list in layers:
    loss_  = tf.nn.l2_loss( xhat_ - x_) #original values of x will be stored in the placeholder
    nmse_  = tf.nn.l2_loss( (xhat_ - x_) ) / nmse_denom_ # computing the nmse.
    if var_list is not None:
            train_ = tf.train.AdamOptimizer(tr_).minimize(loss_, var_list=var_list) 
            training_stages.append( (name,xhat_,loss_,nmse_,train_,var_list) )
    for fm in refinements:
            train2_ = tf.train.AdamOptimizer(tr_*fm).minimize(loss_)
            training_stages.append( (name+' trainrate=' + str(fm) ,xhat_,loss_,nmse_,train2_,()) )
            
if final_refine:
    train2_ = tf.train.AdamOptimizer(tr_*final_refine).minimize(loss_)
    training_stages.append( (name+' final refine ' + str(final_refine) ,xhat_,loss_,nmse_,train2_,()) )

In [38]:
print(*training_stages , sep = "\n")

('Linear trainrate=0.5', <tf.Tensor 'MatMul_1:0' shape=(1600, ?) dtype=float32>, <tf.Tensor 'L2Loss_1:0' shape=() dtype=float32>, <tf.Tensor 'truediv:0' shape=() dtype=float32>, <tf.Operation 'Adam' type=NoOp>, ())
('Linear trainrate=0.1', <tf.Tensor 'MatMul_1:0' shape=(1600, ?) dtype=float32>, <tf.Tensor 'L2Loss_1:0' shape=() dtype=float32>, <tf.Tensor 'truediv:0' shape=() dtype=float32>, <tf.Operation 'Adam_1' type=NoOp>, ())
('Linear trainrate=0.01', <tf.Tensor 'MatMul_1:0' shape=(1600, ?) dtype=float32>, <tf.Tensor 'L2Loss_1:0' shape=() dtype=float32>, <tf.Tensor 'truediv:0' shape=() dtype=float32>, <tf.Operation 'Adam_2' type=NoOp>, ())
('LISTA T=1', <tf.Tensor 'mul:0' shape=(1600, ?) dtype=float32>, <tf.Tensor 'L2Loss_3:0' shape=() dtype=float32>, <tf.Tensor 'truediv_1:0' shape=() dtype=float32>, <tf.Operation 'Adam_3' type=NoOp>, (<tf.Variable 'lam_0:0' shape=() dtype=float32_ref>,))
('LISTA T=1 trainrate=0.5', <tf.Tensor 'mul:0' shape=(1600, ?) dtype=float32>, <tf.Tensor 'L2Los

## Do training LISTA

In [39]:
def save_trainable_vars(sess,filename,**kwargs):
    """save a .npz archive in `filename`  with
    the current value of each variable in tf.trainable_variables()
    plus any keyword numpy arrays.
    """
    save={}
    for v in tf.trainable_variables():
        save[str(v.name)] = sess.run(v)
    save.update(kwargs)
    np.savez(filename,**save)

In [40]:
def load_trainable_vars(sess,filename):
    """load a .npz archive and assign the value of each loaded
    ndarray to the trainable variable whose name matches the
    archive key.  Any elements in the archive that do not have
    a corresponding trainable variable will be returned in a dict.
    """
    other={}
    try:
        tv=dict([ (str(v.name),v) for v in tf.trainable_variables() ])
        for k,d in np.load(filename).items():
            if k in tv:
                print('restoring ' + k)
                sess.run(tf.assign( tv[k], d) )
            else:
                other[k] = d
    except IOError:
        pass
    return other

naming convention:
LISTA_SPARC_layers_rate_awgn_var.npz


In [41]:
ivl, maxit, better_wait =10,1000000,3000
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('norms xval:{xval:.7f} yval:{yval:.7f}'.format(xval=la.norm(beta_val), yval=la.norm(y_val) ) )

state = load_trainable_vars(sess,'LISTA_SPARC_6_R0p5_01.npz') # must load AFTER the initializer

    # must use this same Session to perform all training
    # if we start a new Session, things would replay and we'd be training with our validation set (no no)

done=state.get('done',[])
log=str(state.get('log',''))

for name,xhat_,loss_,nmse_,train_,var_list in training_stages:
    if name in done:
        print('Already did ' + name + '. Skipping.')
        continue
    if len(var_list):
        describe_var_list = 'extending ' + ','.join([v.name for v in var_list])
    else:
        describe_var_list = 'fine tuning all ' + ','.join([v.name for v in tf.trainable_variables() ])

    print(name + ' ' + describe_var_list)
    nmse_history=[]
    for i in range(maxit+1):
        if i%ivl == 0:
            nmse = sess.run(nmse_,feed_dict={y_:y_val,x_:beta_val})
            if np.isnan(nmse):
                raise RuntimeError('nmse is NaN')
            nmse_history = np.append(nmse_history,nmse)
            nmse_dB = 10*np.log10(nmse)
            nmsebest_dB = 10*np.log10(nmse_history.min())
            sys.stdout.write('\ri={i:<6d} nmse={nmse:.6f} dB (best={best:.6f})'.format(i=i,nmse=nmse_dB,best=nmsebest_dB))
            sys.stdout.flush()
            if i%(100*ivl) == 0:
                print('')
                age_of_best = len(nmse_history) - nmse_history.argmin()-1 # how long ago was the best nmse?
                if age_of_best*ivl > better_wait:
                    break # if it has not improved on the best answer for quite some time, then move along
        #y,x = prob(sess)
        y, x= sess.run( ( ygen_,xgen_ ) ) # generates different y,x for every iteration.
        sess.run(train_,feed_dict={y_:y,x_:x} )
    done = np.append(done,name)

    log =  log+'\n{name} nmse={nmse:.6f} dB in {i} iterations'.format(name=name,nmse=nmse_dB,i=i)

    state['done'] = done
    state['log'] = log
    save_trainable_vars(sess,'LISTA_SPARC_6_R0p5_01.npz',**state)

## Testing


In [ ]:
beta_test, beta_test2, n, colsby2_test = gen_msg(2,M, L, K, R, rng)

In [ ]:
y_test = np.matmul(A, beta_test) + np.random.normal( size = (n,colsby2_test),scale= math.sqrt(awgn_var))

In [ ]:
count = 0
for name,xhat_,var_list in layers:
    count = count + 1
    if count != T+1:
        continue
    else:
        out = sess.run(xhat_,feed_dict={y_:y_test,x_:beta_test})
    #if count == 2:
    #    out = sess.run(xhat_,feed_dict={y_:y_test,x_:beta_test})

In [ ]:
print('xtest= \n',beta_test)
print('estimated out \n', out)

xtest= 
 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
estimated out 
 [[ 0.3396565 ]
 [ 0.22745505]
 [-0.6091439 ]
 ...
 [-0.        ]
 [-0.        ]
 [ 0.        ]]


In [ ]:
print(beta_test.sum())
print(out.sum())

50.0
-11.148475


In [ ]:
test_list = range(5)
print(*test_list)

0 1 2 3 4


In [ ]:
test2 = np.array(15)
print(test2.shape)

()


In [ ]:
order0  = np.zeros(test2.shape + (3,), dtype=np.uint32)
print(order0)

[0 0 0]
